# Pandas 数据计算与转换教程

## 课程介绍

在数据分析过程中，数据计算和转换是最常见也是最重要的操作。相比Excel中的公式和数据透视表，Pandas提供了更强大和灵活的数据处理能力。本章节将介绍：

1. 如何进行高效的数据计算
2. 如何使用统计函数进行数据分析
3. 如何进行数据格式的转换和重塑
4. 如何处理复杂的数据透视需求

通过本章节的学习，你将能够：

- 摆脱Excel复杂的公式嵌套
- 实现批量化的数据计算
- 灵活处理各种数据格式的转换
- 深入理解数据透视的原理和应用

让我们首先准备数据：

In [2]:
# 导入所需的库
import pandas as pd
import numpy as np

# 读取数据
df_students = pd.read_csv('data/student_info.csv')
df_orders = pd.read_csv('data/order_info.csv')
df_products = pd.read_csv('data/product_info.csv')
df_order_details = pd.read_csv('data/order_details.csv')


In [3]:
# 转换日期列
df_orders['订单日期'] = pd.to_datetime(df_orders['订单日期'])
df_orders['下单时间'] = pd.to_datetime(df_orders['下单时间'])

# 基础计算操作

在数据分析工作中，数据计算是最基础也是最常见的操作。与Excel相比，Pandas提供了更高效和灵活的计算方式。本节将介绍四种主要的计算操作：算术运算、统计函数、累计计算和自定义函数应用。

## 算术运算

### 原理介绍

Pandas的算术运算是向量化的，这意味着可以直接对整列数据进行运算，无需使用循环。类似于Excel中对整列使用公式，但更加高效和灵活。

### 主要特点：

1. **高效性**：底层使用numpy实现，性能远超Python循环
2. **简洁性**：一行代码即可替代复杂的循环操作
3. **自动对齐**：基于索引自动对齐数据进行计算

### 常见运算：

- 基本运算：+、-、*、/
- 整除：//
- 求余：%
- 幂运算：**

### 代码示例：

In [7]:
# 示例1：基础价格计算
df_orders['成本'] = df_orders['订单金额'] * 0.7  # 计算成本
df_orders['利润'] = df_orders['订单金额'] - df_orders['成本']  # 计算利润

# 示例2：折扣计算
df_orders['折扣比例'] = (df_orders['优惠金额'] / df_orders['订单金额'] * 100).round(2)

# 示例3：多列运算
df_orders['每件商品均价'] = (df_orders['订单金额'] / df_orders['购买数量']).round(2)

print("基础运算结果示例：")
print(df_orders[['订单金额', '成本', '利润', '折扣比例', '每件商品均价']].head())

基础运算结果示例：
      订单金额       成本       利润  折扣比例  每件商品均价
0  1102.11  771.477  330.633  0.00  367.37
1    88.35   61.845   26.505  0.00   29.45
2   108.48   75.936   32.544  0.00   54.24
3    79.72   55.804   23.916  0.09   79.72
4   404.97  283.479  121.491  6.42  134.99


### 注意事项：

1. 运算中含有空值时，结果也会是空值
2. 不同数据类型运算时可能发生强制类型转换
3. 注意除零错误的处理

##  统计函数

### 原理介绍

统计函数提供了对数据进行描述性统计的快捷方式，相当于Excel中的统计函数但使用更便捷。

### 常用统计函数：

1. **基础统计**
   - count(): 非空值计数
   - sum(): 求和
   - mean(): 平均值
   - median(): 中位数

2. **分布统计**
   - std(): 标准差
   - var(): 方差
   - min()/max(): 最小值/最大值
   - quantile(): 分位数

### 代码示例：

In [9]:
# 方法一：使用agg/aggregate方法
order_stats = df_orders['订单金额'].agg({
    '订单数': 'count',
    '总金额': 'sum',
    '平均金额': 'mean',
    '中位数': 'median',
    '标准差': 'std',
    '最小值': 'min',
    '最大值': 'max'
}).round(2)

# 或者方法二：使用describe()方法（更简便）
order_stats = df_orders['订单金额'].describe().round(2)

print("订单金额基本统计：")
print(order_stats)

# 如果要更灵活的统计，可以这样写：
order_stats = df_orders.agg({
    '订单金额': ['count', 'sum', 'mean', 'median', 'std', 'min', 'max'],
    '购买数量': ['sum', 'mean', 'max']
}).round(2)

print("\n多字段统计：")
print(order_stats)

# 分组统计示例
major_stats = df_orders.groupby('专业').agg({
    '订单金额': ['count', 'sum', 'mean'],
    '购买数量': 'sum'
}).round(2)

# 重命名列名使其更易读
major_stats.columns = ['订单数', '订单总额', '平均订单金额', '总购买数量']

print("\n各专业订单统计：")
print(major_stats)

订单金额基本统计：
count     3028.00
mean      1298.72
std       3187.90
min          6.11
25%        114.57
50%        280.36
75%        721.28
max      24901.15
Name: 订单金额, dtype: float64

多字段统计：
              订单金额     购买数量
count      3028.00      NaN
sum     3932526.42  9106.00
mean       1298.72     3.01
median      280.36      NaN
std        3187.90      NaN
min           6.11      NaN
max       24901.15     5.00

各专业订单统计：
       订单数       订单总额   平均订单金额  总购买数量
专业                                   
信息管理   434  556239.77  1281.66   1292
工商管理   418  565460.24  1352.78   1265
市场营销   418  578559.54  1384.11   1249
数据科学   420  581823.30  1385.29   1293
物流管理   423  627252.51  1482.87   1233
电子商务   432  482422.01  1116.72   1294
计算机科学  483  540769.05  1119.60   1480


# Pandas中的agg/aggregate函数详解

## 基本概念

agg (aggregate的简写) 是Pandas中用于数据聚合的高级函数，它可以：

1. 同时对多个列执行不同的统计操作
2. 使用内置或自定义的聚合函数
3. 灵活命名聚合结果
4. 支持分组后的聚合操作

## agg的基本用法

###  单列单个聚合

In [4]:
# 最基础的用法：对单列进行单个统计
result = df_orders['订单金额'].agg('sum')
print("订单总金额:", result)

# 等价写法
result = df_orders['订单金额'].agg(func='sum')
print("订单总金额:", result)

订单总金额: 3932526.42
订单总金额: 3932526.42


单列多个聚合

In [5]:
# 对单列进行多个统计
result = df_orders['订单金额'].agg(['sum', 'mean', 'max', 'min'])
print("\n订单金额的多个统计指标：")
print(result)


订单金额的多个统计指标：
sum     3.932526e+06
mean    1.298721e+03
max     2.490115e+04
min     6.110000e+00
Name: 订单金额, dtype: float64


In [12]:
type(result)

pandas.core.series.Series

多列相同聚合

In [13]:
# 对多个列执行相同的聚合操作
result = df_orders[['订单金额', '购买数量']].agg(['sum', 'mean'])
print("\n多列相同统计指标：")
print(result)


多列相同统计指标：
              订单金额         购买数量
sum   3.932526e+06  9106.000000
mean  1.298721e+03     3.007266


多列不同聚合


In [14]:
# 对不同列执行不同的聚合操作
result = df_orders.agg({
    '订单金额': ['sum', 'mean', 'max'],
    '购买数量': ['sum', 'mean'],
    '评分': ['mean', 'count']
})
print("\n多列不同统计指标：")
print(result)


多列不同统计指标：
               订单金额         购买数量          评分
sum    3.932526e+06  9106.000000         NaN
mean   1.298721e+03     3.007266    2.981763
max    2.490115e+04          NaN         NaN
count           NaN          NaN  987.000000


agg使用自定义函数

In [6]:
# 首先导入必要的库
import pandas as pd
import numpy as np

# 1.1 最简单的自定义聚合函数
def value_range(x):
    """计算数值范围"""
    return x.max() - x.min()

# 使用方式
result1 = df_orders['订单金额'].agg(value_range)
print("数值范围:", result1)
#自定义的函数作为参数传入
# 1.2 返回多个统计值的函数
def order_summary(x):
    """返回一个包含多个统计值的Series"""
    return pd.Series({
        '总和': x.sum(),
        '平均值': x.mean(),
        '最大值': x.max(),
        '最小值': x.min(),
        '数据范围': x.max() - x.min(),
        '标准差': x.std(),
        '样本数': len(x)
    })

# 使用方式
result2 = df_orders['订单金额'].agg(order_summary)
print("\n订单汇总统计：")
print(result2)

数值范围: 24895.04

订单汇总统计：
总和      3.932526e+06
平均值     1.298721e+03
最大值     2.490115e+04
最小值     6.110000e+00
数据范围    2.489504e+04
标准差     3.187899e+03
样本数     3.028000e+03
dtype: float64


数据透视表（Pivot Table）是一种数据分析工具，用于快速汇总、分类和整理大量数据，帮助你从不同角度查看和理解数据。它可以将原始数据转换成有意义的报告，而不需要手动进行复杂的计算。

### 通俗解释
想象你有一大堆详细的销售记录，包括销售人员、销售产品、销售日期、销售额等。数据透视表可以帮你快速回答这些问题，比如：
- **每个销售人员的总销售额是多少？**
- **每种产品在不同时间段的销售额如何？**
- **不同地区的销售表现如何？**

### 数据透视表是怎么工作的？
你可以把数据透视表想象成一个灵活的电子表格，它允许你“旋转”和重新排列数据，从不同角度来查看和分析。例如：
- **行（Row）**：你可以将某个字段（如销售人员或产品）拖到行区域，数据透视表会按这个字段分类显示数据。
- **列（Column）**：你可以将另一个字段（如销售年份或月份）拖到列区域，数据透视表会按这个字段进一步分类显示数据。
- **值（Values）**：你可以将数值字段（如销售额）拖到值区域，数据透视表会自动计算总和、平均值或其他汇总结果。
- **筛选（Filter）**：你可以对某个字段进行筛选，只显示你感兴趣的部分数据。

### 举个简单的例子
假设你有以下销售数据：

| 销售人员 | 产品   | 销售额 | 销售地区 |
|--------|-------|------|--------|
| 张三   | 手机   | 5000 | 北京    |
| 李四   | 电脑   | 7000 | 上海    |
| 张三   | 电脑   | 8000 | 北京    |
| 李四   | 手机   | 6000 | 上海    |

**用数据透视表可以很快回答这些问题：**
- **每个销售人员的总销售额是多少？**
  - 数据透视表会显示张三的总销售额是 13000 元，李四的总销售额是 13000 元。
- **不同产品的销售额是多少？**
  - 数据透视表会显示手机的总销售额是 11000 元，电脑的总销售额是 15000 元。

### 数据透视表的好处
- **节省时间**：你可以快速生成各种报告，而不需要手动汇总或筛选数据。
- **灵活性**：可以随时调整和重新排列数据，以不同方式查看信息。
- **直观**：让你更清晰地理解数据背后的趋势和模式。

总之，数据透视表是一个非常强大的数据分析工具，特别适合用来处理和分析大规模数据。

# 数据重塑：Excel透视表的进阶版

在Excel中，我们经常使用数据透视表来汇总和分析数据。Pandas提供了更强大和灵活的数据重塑功能，让我们能够实现更复杂的数据分析。

## 1. 宽表转长表（melt）

相当于Excel中的"取消数据透视表"操作，但更加灵活。

### 场景演示：

假设我们有一个按月份展开的销售数据表（类似Excel中横向展开的月度数据）

In [20]:
# 首先创建一个宽表示例（类似Excel中的横向数据）
monthly_sales = df_orders.pivot_table(
    index='专业',
    columns=df_orders['订单日期'].dt.to_period('M'),
    values='订单金额',
    aggfunc='sum'
).round(2)

print("原始宽表格式（类似Excel中横向展示的月度数据）：")
print(monthly_sales)

# 转换为长表格式（类似Excel中的明细数据）
long_format = monthly_sales.melt(
    ignore_index=False,
    var_name='月份',     
    value_name='销售额'  
)

print("\n转换后的长表格式（类似Excel中的竖向数据）：")
print(long_format)

原始宽表格式（类似Excel中横向展示的月度数据）：
订单日期    2023-11   2023-12   2024-01   2024-02   2024-03   2024-04   2024-05  \
专业                                                                            
信息管理   43676.33  59593.77  45578.17  41135.02  68941.80  64959.57  41980.36   
工商管理   27915.15  41014.17  51635.00  41442.01  33521.18  36332.57  36165.84   
市场营销   38290.83  55727.03  20737.00  55638.84  65791.67  55084.39  64424.84   
数据科学   14463.78  57534.27  70765.14  28593.38  33918.11  48473.62  59405.50   
物流管理   45543.72  73085.44  61513.67  47205.34  76263.58  36268.07  77731.01   
电子商务   13845.15  28309.78  37909.66  37498.87  54391.46  25786.24  69512.65   
计算机科学  30531.37  48843.22  46765.20  37793.04  54346.41  97401.54  30895.39   

订单日期    2024-06   2024-07   2024-08   2024-09   2024-10   2024-11  
专业                                                                 
信息管理   18765.72  27767.06  27177.46  42569.20  33903.75  40191.56  
工商管理   66153.27  37304.60  80223.30  71038.51  32719.52  

In [7]:
# 类似Excel中创建一个简单的数据透视表
basic_pivot = df_orders.pivot_table(
     index='专业',        # Excel中的行标签
    columns='年级',      # Excel中的列标签
    values='订单金额',   # Excel中的值
    aggfunc='sum'       # Excel中的汇总方式
)
#也可以换一种写法
basic_pivot1 = pd.pivot_table(
    df_orders,
    index='专业',        # Excel中的行标签
    columns='年级',      # Excel中的列标签
    values='订单金额',   # Excel中的值
    aggfunc='sum'       # Excel中的汇总方式
)
print("基础透视表（类似Excel简单透视表）：")
print(basic_pivot)
print("第二种写法")
print(basic_pivot1)

基础透视表（类似Excel简单透视表）：
年级            大一         大三         大二         大四
专业                                               
信息管理   176319.09   85474.91  138618.26  155827.51
工商管理   114008.62  178643.67  119090.93  153717.02
市场营销   247607.34   90124.04   80764.88  160063.28
数据科学   171047.14  118527.21  103220.57  189028.38
物流管理   101942.49  113326.21  169515.89  242467.92
电子商务   102401.74   93819.41  121408.16  164792.70
计算机科学   98300.78  216603.31   86411.83  139453.13
第二种写法
年级            大一         大三         大二         大四
专业                                               
信息管理   176319.09   85474.91  138618.26  155827.51
工商管理   114008.62  178643.67  119090.93  153717.02
市场营销   247607.34   90124.04   80764.88  160063.28
数据科学   171047.14  118527.21  103220.57  189028.38
物流管理   101942.49  113326.21  169515.89  242467.92
电子商务   102401.74   93819.41  121408.16  164792.70
计算机科学   98300.78  216603.31   86411.83  139453.13


高级透视表（相当于Excel中的复杂数据透视表）

In [8]:
advanced_pivot = pd.pivot_table(
    df_orders,
    index=['专业', '年级'],      # 行标签
    columns='商品类别',          # 列标签
    values=['订单金额', '购买数量'],  # 值
    aggfunc=['sum', 'mean'],    # 使用字符串而不是np.sum, np.mean
    margins=True,               # 显示总计
    margins_name='总计'
).round(2)

print("\n高级透视表（使用字符串指定聚合函数）：")
print(advanced_pivot)


高级透视表（使用字符串指定聚合函数）：
                sum                                                         \
               订单金额                                                          
商品类别             图书      学习用品         服装       生活用品        电子产品       运动器材   
专业    年级                                                                     
信息管理  大一    3951.96    818.55    9333.45    6230.74   139149.71   14699.56   
      大三    5825.83   1296.71    9173.38    4996.65    44059.05   16648.59   
      大二    4445.37    946.16    5209.72    3944.09   102721.00   19795.88   
      大四    3529.75    882.17    6963.59    5323.73   127011.47   10293.60   
工商管理  大一    1915.45    916.78    3326.31    3520.49    93172.48    9172.11   
      大三    2914.63   1690.00    8625.62    4918.25   147608.36   11139.94   
      大二    2709.21   1498.82    8765.75    6867.91    89574.28    8624.65   
      大四    4821.72    954.59   13580.14    7167.56   106795.21   16893.42   
市场营销  大一    4674.78   1402.22   10881.84   

### Excel对应操作说明：

1. **基本对应关系**：
   - index = Excel中的"行标签"
   - columns = Excel中的"列标签"
   - values = Excel中的"值"
   - aggfunc = Excel中的"汇总方式"
   - margins = Excel中的"总计"功能

2. **与Excel的区别**：
   - Excel中重复使用同一字段作为值时需要多次拖拽
   - Excel中每个值字段只能选择一种汇总方式
   - Pandas可以同时对不同字段使用不同的汇总方式

3. **实际应用场景**：

In [36]:
# 创建一个销售分析报表（类似Excel中常见的销售分析透视表）
sales_analysis = pd.pivot_table(
    df_orders,  # 使用原始数据框df_orders
    index=['专业', '年级'],  # 将'专业'和'年级'作为行索引，分组分析
    columns=['商品类别'],  # 将'商品类别'作为列索引，展开不同商品类别
    values=['订单金额', '购买数量'],  # 要分析的值为'订单金额'和'购买数量'
    aggfunc={
        '订单金额': 'sum',  # 对'订单金额'列进行求和，计算每个分组的总销售金额
        '购买数量': 'mean'  # 对'购买数量'列进行求平均，计算每个分组的平均购买数量
    },
    margins=True,  # 添加总计行和总计列，显示整体的汇总数据
    fill_value=0  # 将缺失值填充为0，避免数据为空
).round(2)  # 将结果四舍五入到小数点后两位

# 重命名列，使列名更易于阅读，使用f字符串格式
sales_analysis.columns = [f'{col[0]}_{col[1]}' for col in sales_analysis.columns]

print("销售分析报表：")  # 输出销售分析报表的标题
print(sales_analysis)  # 打印销售分析报表


销售分析报表：
            订单金额_图书  订单金额_学习用品    订单金额_服装  订单金额_生活用品   订单金额_电子产品  订单金额_运动器材  \
专业    年级                                                                      
信息管理  大一    3951.96     818.55    9333.45    6230.74   139149.71   14699.56   
      大三    5825.83    1296.71    9173.38    4996.65    44059.05   16648.59   
      大二    4445.37     946.16    5209.72    3944.09   102721.00   19795.88   
      大四    3529.75     882.17    6963.59    5323.73   127011.47   10293.60   
工商管理  大一    1915.45     916.78    3326.31    3520.49    93172.48    9172.11   
      大三    2914.63    1690.00    8625.62    4918.25   147608.36   11139.94   
      大二    2709.21    1498.82    8765.75    6867.91    89574.28    8624.65   
      大四    4821.72     954.59   13580.14    7167.56   106795.21   16893.42   
市场营销  大一    4674.78    1402.22   10881.84    8049.54   208069.25   11979.22   
      大三    2799.57     960.10    8964.70    1758.72    66871.75    7236.27   
      大二    3159.63    1148.98    8047.16   

在 Pandas 中，stack 和 unstack 操作主要用于将数据框的行和列之间进行转换。它们类似于 Excel 中的“透视表”操作，具体如下：

stack：将数据框的列索引“压缩”到行索引中，通常会将宽格式的数据转换为长格式。这类似于在 Excel 中将数据从列的布局转换成按行展示的布局。

unstack：与 stack 相反，将行索引中的部分数据“解压”成列索引，通常将长格式的数据转换为宽格式。这类似于在 Excel 中将按行组织的数据转变成按列显示的形式。

示例
在 Excel 中，stack 操作可以类比为将多列的值“折叠”到一个列中，而 unstack 操作则可以类比为将一个列的值“展开”到多列中，就像在使用数据透视表时将字段拖动到行或列的位置。

In [35]:
# 使用stack方法将列索引转为行索引，并使用future_stack=True参数以适应未来的Pandas版本
stacked = advanced_pivot.stack(future_stack=True)
print("Stack后的数据：")  # 输出Stack后的数据
print(stacked.head())  # 打印前几行的数据

# 使用unstack方法将行索引转为列索引，将长格式数据恢复为宽格式
unstacked = stacked.unstack()
print("\nUnstack后的数据：")  # 输出Unstack后的数据
print(unstacked.head())  # 打印前几行的数据

# 4.2 创建具有多级索引的数据透视表
multi_index = pd.pivot_table(
    df_orders,  # 原始数据框
    index=['专业', '年级'],  # 将'专业'和'年级'设置为行索引
    columns=['商品类别', '支付方式'],  # 将'商品类别'和'支付方式'设置为列索引
    values='订单金额',  # 使用'订单金额'列的值进行聚合
    aggfunc='sum'  # 聚合函数为求和
).round(2)  # 将结果四舍五入到小数点后两位

# 使用stack方法指定某个索引级别进行转换
level_stacked = multi_index.stack(level=0, future_stack=True)  # 将第一个级别的列索引转为行索引
print("\n指定level stack后的数据：")  # 输出指定level stack后的数据
print(level_stacked.head())  # 打印前几行的数据




Stack后的数据：
                    sum           mean      
                   订单金额  购买数量     订单金额  购买数量
专业   年级 商品类别                                
信息管理 大一 图书      3951.96  46.0   188.19  2.19
        学习用品     818.55  36.0    74.41  3.27
        服装      9333.45  63.0   424.25  2.86
        生活用品    6230.74  57.0   366.51  3.35
        电子产品  139149.71  44.0  9276.65  2.93

Unstack后的数据：
             sum                                                           \
            订单金额                                                            
商品类别          图书     学习用品       服装     生活用品       电子产品      运动器材       食品   
专业   年级                                                                     
信息管理 大一  3951.96   818.55  9333.45  6230.74  139149.71  14699.56  2135.12   
     大三  5825.83  1296.71  9173.38  4996.65   44059.05  16648.59  3474.70   
     大二  4445.37   946.16  5209.72  3944.09  102721.00  19795.88  1556.04   
     大四  3529.75   882.17  6963.59  5323.73  127011.47  10293.60  1823.20   